In [1]:
def getUserPurchaseHistory(user_id):
    transaction_data = pd.read_csv("data/user_purchase_history.csv")
    user_transaction_data = transaction_data[(transaction_data['User'] == user_id)].copy()
    return user_transaction_data

In [2]:
import numpy as np
import random
import datetime
import pandas as pd
from sys_constant import aging_criteria, inactive, item_dict, user_dict
from sklearn.linear_model import LinearRegression
from feature_engine.categorical_encoders import OneHotCategoricalEncoder
from sklearn.model_selection import train_test_split

from flask import Flask, request, make_response

import re
from flask import jsonify

import user_things as ut
from recommendation_helper import RecommenderEngine
from recommendation_helper import AddProduct , UserModel
from sys_constant import ignored, today
from stereotype import Stereo

In [3]:
user_id = 10000
purchase_pattern_data = pd.DataFrame()
user_transaction_data = getUserPurchaseHistory(int(user_id))
item_list = user_transaction_data.Product.unique().tolist()
for item in item_list:
    item_data = user_transaction_data[(user_transaction_data['Product'] == item)].copy()
    size_list = item_data.Size.unique().tolist()
    for size in size_list:
        tempdataset = item_data[(item_data['Size'] == size)].copy()
        if tempdataset.empty == True:
            continue
        else:
            tempdataset['Transaction_Date'] = pd.to_datetime(tempdataset.Transaction_Date)
            tempdataset.sort_values(by=['Transaction_Date'], inplace=True, ascending=True)
            #sys_date = datetime.datetime.now().date()
            sys_date = datetime.datetime.strptime('2019-11-27','%Y-%m-%d').date()
            last_purchase_date = tempdataset.Transaction_Date.iloc[-1]
            last_purchase_date = last_purchase_date.date()
            days_diff = (sys_date - last_purchase_date).days
            if (days_diff > aging_criteria and len(tempdataset) == 1):
                frequent_item = False
                purchase_pattern_data = purchase_pattern_data.append({'User': user_id,
                                                                      'Product': item,
                                                                      'Product_Name': item_dict[item],
                                                                      'Size': size,
                                                                      'Purchase_Freq': -3,
                                                                      'Preferred_Shopping_Weekday': -3,
                                                                      'Preferred_Store': tempdataset.Store,
                                                                      'Last_Purchased': last_purchase_date,
                                                                      'Days_Left': -2,
                                                                      'Approx_Item_Price': tempdataset.Price,
                                                                      'Frequent_Item': frequent_item,
                                                                      'Reminders': inactive,},
                                                                     ignore_index=True)
                continue
            else:
                reminders = 0
                tempdataset['Frequency'] = tempdataset.Transaction_Date.diff()
                freqdata = tempdataset.Frequency.value_counts().index[0] # gets mode
                purchase_freq = freqdata.days
                if (days_diff <= purchase_freq):
                    days_left = purchase_freq - days_diff
                    frequent_item = True
                elif days_diff / purchase_freq <= 2:
                    days_left = -1
                    frequent_item = True
                    reminders = days_diff // purchase_freq
                else:
                    days_left = -2
                    frequent_item = False
                    reminders = inactive
                freq_store = tempdataset.Store.value_counts().index[0]
                freq_shopping_day = tempdataset.WeekDay.value_counts().index[0]
                price = tempdataset.Price.mean()
                purchase_pattern_data = purchase_pattern_data.append({'User': user_id,
                                                                      'Product': item,
                                                                      'Product_Name': item_dict[item],
                                                                      'Size': size,
                                                                      'Purchase_Freq': purchase_freq,
                                                                      'Preferred_Shopping_Weekday': freq_shopping_day,
                                                                      'Preferred_Store': freq_store,
                                                                      'Last_Purchased': last_purchase_date,
                                                                      'Days_Left': days_left,
                                                                      'Approx_Item_Price': round(price,2),
                                                                      'Frequent_Item': frequent_item,
                                                                      'Reminders': reminders,}, ignore_index=True)

In [4]:
purchase_pattern_data.head(40)

,Approx_Item_Price,Days_Left,Frequent_Item,Last_Purchased,Preferred_Shopping_Weekday,Preferred_Store,Product,Product_Name,Purchase_Freq,Reminders,Size,User
0,5.05,-2.0,0.0,2019-10-28,Friday,Tesco,7009.0,Sugar,4.0,Inactive,M,10000.0
1,2.12,-1.0,1.0,2019-11-21,Monday,Aldi,7009.0,Sugar,4.0,1,S,10000.0
2,7.15,9.0,1.0,2019-11-19,Tuesday,Tesco,7009.0,Sugar,17.0,0,L,10000.0
3,7.21,-1.0,1.0,2019-11-21,Tuesday,Aldi,7003.0,Butter,3.0,2,L,10000.0
4,1.98,-2.0,0.0,2019-08-20,Friday,Lidl,7003.0,Butter,21.0,Inactive,S,10000.0
5,5.13,19.0,1.0,2019-05-17,Friday,Spar,7003.0,Butter,213.0,0,M,10000.0
6,7.31,-1.0,1.0,2019-11-19,Tuesday,Tesco,7006.0,Chicken,4.0,2,L,10000.0
7,5.08,-2.0,0.0,2019-11-09,Friday,Tesco,7006.0,Chicken,6.0,Inactive,M,10000.0
8,2.25,-1.0,1.0,2019-11-21,Friday,Aldi,7006.0,Chicken,3.0,2,S,10000.0
9,5.13,-2.0,0.0,2019-11-09,Tuesday,Aldi,7014.0,Cookies,3.0,Inactive,M,10000.0


In [5]:
buy_now = purchase_pattern_data[(purchase_pattern_data.Days_Left == today)]
buy_now.head()

,Approx_Item_Price,Days_Left,Frequent_Item,Last_Purchased,Preferred_Shopping_Weekday,Preferred_Store,Product,Product_Name,Purchase_Freq,Reminders,Size,User
11,2.17,0.0,1.0,2019-11-19,Tuesday,Tesco,7014.0,Cookies,8.0,0,S,10000.0
14,1.88,0.0,1.0,2019-11-21,Tuesday,Aldi,7002.0,Bread,6.0,0,S,10000.0
22,7.14,0.0,1.0,2019-11-21,Wednesday,Aldi,7011.0,Rice,6.0,0,L,10000.0


In [6]:
buy_later = purchase_pattern_data[(purchase_pattern_data.Days_Left > today)]
buy_later.head()

,Approx_Item_Price,Days_Left,Frequent_Item,Last_Purchased,Preferred_Shopping_Weekday,Preferred_Store,Product,Product_Name,Purchase_Freq,Reminders,Size,User
2,7.15,9.0,1.0,2019-11-19,Tuesday,Tesco,7009.0,Sugar,17.0,0,L,10000.0
5,5.13,19.0,1.0,2019-05-17,Friday,Spar,7003.0,Butter,213.0,0,M,10000.0
24,6.28,229.0,1.0,2019-07-14,Sunday,Aldi,7016.0,Cake,365.0,0,L,10000.0


In [7]:
ignored = purchase_pattern_data[(purchase_pattern_data.Days_Left < today)]
ignored.head()

,Approx_Item_Price,Days_Left,Frequent_Item,Last_Purchased,Preferred_Shopping_Weekday,Preferred_Store,Product,Product_Name,Purchase_Freq,Reminders,Size,User
0,5.05,-2.0,0.0,2019-10-28,Friday,Tesco,7009.0,Sugar,4.0,Inactive,M,10000.0
1,2.12,-1.0,1.0,2019-11-21,Monday,Aldi,7009.0,Sugar,4.0,1,S,10000.0
3,7.21,-1.0,1.0,2019-11-21,Tuesday,Aldi,7003.0,Butter,3.0,2,L,10000.0
4,1.98,-2.0,0.0,2019-08-20,Friday,Lidl,7003.0,Butter,21.0,Inactive,S,10000.0
6,7.31,-1.0,1.0,2019-11-19,Tuesday,Tesco,7006.0,Chicken,4.0,2,L,10000.0


## Shop Recommender

In [10]:
transaction_data = pd.read_csv("data/user_purchase_history.csv")
store_list = transaction_data.Store.unique().tolist()
item_list = transaction_data.Product.unique().tolist()
shop_recommender = pd.DataFrame(
columns=['Product', 'Size', 'Store', 'Avg_Price', 'Best_Price', 'Best_Price_Updated_On'])
for product in item_list:
    product_data = transaction_data[(transaction_data.Product == product)]
    size_list = product_data.Size.unique().tolist()
    # print(len(sizelist))
    for size in size_list:
        size_data = product_data[(product_data.Size == size)]
        store_list = size_data.Store.unique().tolist()
        for store in store_list:
            tempdataset = size_data[(size_data['Store'] == store)]
            tempdataset['Transaction_Date'] = pd.to_datetime(tempdataset.Transaction_Date)
            tempdataset.sort_values(by=['Transaction_Date'], inplace=True, ascending=True)
            last_purchased_price = tempdataset.Price.iloc[-1]
            last_purchased_date = tempdataset.Transaction_Date.iloc[-1]
            mean_price = tempdataset.Price.mean()
            shop_recommender = shop_recommender.append({'Product': item_dict[int(product)],
                                                        'Size': size,
                                                        'Store': store,
                                                        'Avg_Price': round(mean_price,2),
                                                        'Best_Price': last_purchased_price,
                                                        'Best_Price_Updated_On': last_purchased_date.date()},
                                                       ignore_index=True)
            shop_recommender = shop_recommender.loc[shop_recommender.groupby(['Product', 'Size'])['Best_Price'].idxmin()]

/home/rhofix/anaconda3/envs/rhofix/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/home/rhofix/anaconda3/envs/rhofix/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [11]:
shop_recommender

,Product,Size,Store,Avg_Price,Best_Price,Best_Price_Updated_On
0,Beef,L,Lidl,7.21,7.63,2019-11-19
1,Beef,M,Aldi,5.01,4.75,2019-11-22
2,Beef,S,Aldi,2.21,0.93,2019-11-21
3,Bleach,L,Tesco,7.32,6.35,2019-11-22
4,Bleach,M,Tesco,4.99,4.51,2019-11-22
5,Bleach,S,Aldi,2.25,1.03,2019-11-21
6,Bread,L,Tesco,7.22,6.12,2019-11-23
7,Bread,M,Aldi,5.01,4.29,2019-11-22
8,Bread,S,M&S,2.21,2.30,2019-11-20
9,Butter,L,Spar,7.25,6.51,2019-11-22


In [12]:
cheapest_milk = shop_recommender[(shop_recommender['Product'] == 'Milk')]
cheapest_milk.head()

,Product,Size,Store,Avg_Price,Best_Price,Best_Price_Updated_On
31,Milk,L,Lidl,7.23,6.53,2019-11-18
32,Milk,M,M&S,5.00,4.71,2019-11-23
33,Milk,S,Aldi,2.24,1.08,2019-11-18


In [13]:
cheapest_milk = shop_recommender[(shop_recommender['Product'] == 'Sugar')]
cheapest_milk.head()

,Product,Size,Store,Avg_Price,Best_Price,Best_Price_Updated_On
46,Sugar,L,Aldi,7.23,7.05,2019-11-22
47,Sugar,M,Tesco,5.04,4.12,2019-11-23
48,Sugar,S,Lidl,2.25,2.47,2019-11-21


In [14]:
user_purchase_data = getUserPurchaseHistory(int(10000))
productlist = user_purchase_data.Product.unique().tolist()
traindata = pd.DataFrame()
for product in productlist:
    productdata = user_purchase_data[(user_purchase_data.Product == product)]
    sizelist = productdata.Size.unique().tolist()
    for size in sizelist:
        tempdata =  productdata[(productdata.Size == size)]
        tempdata['Transaction_Date'] = pd.to_datetime(tempdata.Transaction_Date)
        tempdata = tempdata.sort_values(by=['Transaction_Date'], ascending=True)
        tempdata['Frequency'] = tempdata.Transaction_Date.diff()
        traindata = traindata.append(tempdata)
traindata.drop(['User', 'Store', 'Transaction_Date', 'Price','WeekDay'], axis = 1, inplace=True)
traindata = traindata.dropna()
traindata.Product = traindata.Product.astype(str)
target = traindata.Frequency.dt.days
traindata.drop(['Frequency'], axis = 1, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(traindata, target, test_size=0.1,
                                            random_state=0)
ohe = OneHotCategoricalEncoder(top_categories=None,
                               variables=['Product', 'Size'],
                               drop_last=True)
ohe.fit(X_train, y_train)
X_train = ohe.transform(X_train)
X_test = ohe.transform(X_test)

regressor = LinearRegression()
regressor.fit(X_train, np.log(y_train))
#xgb_model = xgb.XGBRegressor()
#eval_set = [(X_test, np.log(y_test))]
#xgb_model.fit(X_train, np.log(y_train), eval_set=eval_set, verbose=False)

user_purchase_model = pd.DataFrame()
user_testdata = pd.DataFrame()
user_testdata['Product'] = user_purchase_data.Product
user_testdata['Size'] = user_purchase_data.Size
user_testdata.Product = user_testdata.Product.astype(str)
testdata = ohe.transform(user_testdata)
pred = regressor.predict(testdata)
pred = np.exp(pred)
user_purchase_model = user_testdata.copy()
user_purchase_model["Learned Frequencies"] = np.around(pred)
#user_purchase_model = user_purchase_model.groupby(['Product', 'Size', 'Learned Frequencies'])
user_purchase_model = user_purchase_model.drop_duplicates()
userproductlist = user_purchase_model.Product.unique().tolist()
for product in userproductlist:
    user_purchase_model.Product = user_purchase_model.Product.replace(product,item_dict[int(product)])


/home/rhofix/anaconda3/envs/rhofix/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [15]:
grp = user_purchase_model.groupby('Product')
#for key, item in grp:
#    print(grp.get_group(key), "\n\n")

user_purchase_model.sort_values(by=['Product'], inplace=True, ascending=True)
user_purchase_model

,Product,Size,Learned Frequencies
5,Bread,M,8.0
33,Bread,S,7.0
11,Bread,L,7.0
1,Butter,L,5.0
22,Butter,S,5.0
144,Butter,M,5.0
969,Cake,L,365.0
2,Chicken,L,7.0
4,Chicken,M,7.0
6,Chicken,S,7.0


In [16]:
purchase_pattern_data = purchase_pattern_data[['Product_Name','Size','Purchase_Freq']]
purchase_pattern_data.sort_values(by=['Product_Name'], inplace=True, ascending=True)
purchase_pattern_data

,Product_Name,Size,Purchase_Freq
12,Bread,M,4.0
14,Bread,S,6.0
13,Bread,L,3.0
3,Butter,L,3.0
4,Butter,S,21.0
5,Butter,M,213.0
24,Cake,L,365.0
7,Chicken,M,6.0
8,Chicken,S,3.0
6,Chicken,L,4.0


In [18]:
print("Total Transactions in the dataset: ", len(user_transaction_data))
#Transaction Data for User: 10000
user_transaction_data

Total Transactions in the dataset:  1138


,User,Store,Product,Size,Transaction_Date,WeekDay,Price
0,10000,Tesco,7009,M,2019-01-28,Monday,4.99
1,10000,Tesco,7003,L,2019-04-14,Sunday,6.19
2,10000,Tesco,7006,L,2019-04-14,Sunday,7.68
3,10000,Tesco,7014,M,2018-10-12,Friday,4.45
4,10000,Tesco,7006,M,2018-10-12,Friday,4.75
...,...,...,...,...,...,...,...
1133,10000,Tesco,7008,S,2019-07-15,Monday,1.07
1134,10000,Tesco,7008,S,2018-10-12,Friday,0.94
1135,10000,Tesco,7006,M,2019-08-12,Monday,5.01
1136,10000,Tesco,7003,L,2019-07-21,Sunday,6.31
